# Seattle AirBNB Data Analysis

## Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
# import re for regex library
import re


%matplotlib inline

#pd.set_option('display.float_format', lambda x: '%.0f' % x)

### Importing the Dataset

In [2]:
#listings = pd.read_csv("listings.csv", parse_dates=["host_since", "first_review", "last_review"])
df = pd.read_csv("listings.csv")
pd.options.display.max_columns = None
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazi

## We will remove the '_url' columns.

In [3]:
# Remove the columns with urls
print(df.shape)
# record columns to delete
to_del = df.filter(like='_url').columns
print(to_del)
# drop unless columns
df.drop(to_del, axis=1, inplace=True)
print(df.shape)   

(3818, 92)
Index(['listing_url', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url'],
      dtype='object')
(3818, 84)


In [4]:
df.head()

,id,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,16708587,Jill,2014-06-12,"Seattle, Washington, United States",i love living in Seattle. i grew up in the mi...,within a few hours,67%,100%,f,Queen Anne,2.0,2.0,"['email', 'phone', 'google', 'reviews', 'jumio']",t,t,"West Lee Street, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.629724,-122.369483,t,House,Entire home/apt,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV

## Replace Missing Values
A good strategy when dealing with missing values involves their replacement with another value. Usually, the following strategies are adopted:

* for numerical values replace the missing value with the average value of the column
* for categorial values replace the missing value with the most frequent value of the column
* use other functions

In order to replace missing values, three functions can be used: `fillna()`, `replace()` and `interpolate()`. The `fillna()` function replaces all the NaN values with the value passed as argument. For example, for numerical values, all the NaN values in the numeric columns could be replaced with the average value. In order to list the type of a column, we can use the attribute `dtypes` as follows:

### get count of missing values

In [5]:
(df.isna().sum()).sort_values(ascending=False)

license             3818
square_feet         3721
monthly_price       2301
security_deposit    1952
weekly_price        1809
                    ... 
bed_type               0
amenities              0
price                  0
guests_included        0
accommodates           0
Length: 84, dtype: int64

### get percentage of missing values

In [6]:
(df.isna().sum()/len(df)*100).sort_values(ascending=False)

license             100.000000
square_feet          97.459403
monthly_price        60.267156
security_deposit     51.126244
weekly_price         47.380828
                       ...    
bed_type              0.000000
amenities             0.000000
price                 0.000000
guests_included       0.000000
accommodates          0.000000
Length: 84, dtype: float64

### Drop all columns missing more than 80% of their values

In [7]:
print(df.shape)
df.dropna(thresh=0.2*len(df), axis=1,inplace=True)
print(df.shape)

(3818, 84)
(3818, 82)


## Data Formatting

### Incorrect Data Types

In [8]:

df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']] = df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']].replace({'t': True, 'f': False})

# Convert them to `boolean` data types
df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']] = df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']].astype(dtype='bool')

In [9]:
#The price columns are showing as objects or strings

# Remove $ and convert 'price' to float
#listings_price_cols = listings.filter(like='price').columns
df["price"] = df["price"].str.replace('$', '', regex=True)
df["price"] = pd.to_numeric(df["price"], errors='coerce');

df["weekly_price"] = df["weekly_price"].str.replace('$', '', regex=True)
df["weekly_price"] = pd.to_numeric(df["weekly_price"], errors='coerce');

df["monthly_price"] = df["monthly_price"].str.replace('$', '', regex=True)
df["monthly_price"] = pd.to_numeric(df["monthly_price"], errors='coerce');

#listings_deposit_fee_cols = listings[["security_deposit","cleaning_fee"]]
df["security_deposit"] = df["security_deposit"].str.replace('$', '', regex=True)
df["security_deposit"] = pd.to_numeric(df["security_deposit"], errors='coerce');

df["cleaning_fee"] = df["cleaning_fee"].str.replace('$', '', regex=True)
df["cleaning_fee"] = pd.to_numeric(df["cleaning_fee"], errors='coerce');

df["extra_people"] = df["extra_people"].str.replace('$', '', regex=True)
df["extra_people"] = pd.to_numeric(df["extra_people"], errors='coerce');

df["host_response_rate"] = df["host_response_rate"].str.replace('%', '', regex=True)
df["host_response_rate"] = pd.to_numeric(df["host_response_rate"], errors='coerce');

df["host_acceptance_rate"] = df["host_acceptance_rate"].str.replace('%', '', regex=True)
df["host_acceptance_rate"] = pd.to_numeric(df["host_acceptance_rate"], errors='coerce');

#listings.filter(like='price').columns

## Numeric columns

In [10]:
import numpy as np
numeric = df.select_dtypes(include=np.number)
numeric_columns = numeric.columns

Then, we fill the NaN values of numeric columns with the average value, given by the `df.mean()` function.

In [11]:
df[numeric_columns] = df[numeric_columns].fillna(df.mean())

In [12]:
df["price"] = df["price"].astype('int64')
df["weekly_price"] = df["weekly_price"].astype('int64')
df["monthly_price"] = df["monthly_price"].astype('int64')
df["security_deposit"] = df["security_deposit"].astype('int64')
df["cleaning_fee"] = df["cleaning_fee"].astype('int64')
df["extra_people"] = df["extra_people"].astype('int64')

Now, we can check whether the NaN values in numeric columns have been removed.

In [13]:
#(df.isna().sum()/len(df)*100).sort_values(ascending=False)
df[numeric_columns].isnull().sum()

id                                0
scrape_id                         0
host_id                           0
host_response_rate                0
host_acceptance_rate              0
host_listings_count               0
host_total_listings_count         0
latitude                          0
longitude                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
price                             0
weekly_price                      0
monthly_price                     0
security_deposit                  0
cleaning_fee                      0
guests_included                   0
extra_people                      0
minimum_nights                    0
maximum_nights                    0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
review_scores_rating        

## Categorical columns

We note that in dtypes the categorial columns are described as objects. Thus we can select the object columns. We would like to consider only boolean columns. However the object type includes also the column class, which is a string. We select all the object columns, and then we remove from them the column class. Then we can convert the type of the result to bool.

Now we can replace all the missing values for booleans with the most frequent value. We can use the `mode()` function to calculate the most frequent value. We use the `fillna()` function to replace missing values, but we could use also the `replace(old_value,new_value)` function.

In [14]:
boolean_columns = df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']].columns.to_list()
# Cannot fill with mode because of nulls
boolean_columns = df[boolean_columns].fillna(df.mode())
#boolean_columns = df[boolean_columns].interpolate(method='pad')

In [15]:
#df[['host_is_superhost','host_has_profile_pic','host_identity_verified']].dropna()

In [16]:
df[['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license','instant_bookable','require_guest_profile_picture','require_guest_phone_verification']].isna().sum()

host_is_superhost                   0
host_has_profile_pic                0
host_identity_verified              0
is_location_exact                   0
has_availability                    0
requires_license                    0
instant_bookable                    0
require_guest_profile_picture       0
require_guest_phone_verification    0
dtype: int64

In [17]:
print(df.shape)
# Get number of unique values in each column
counts = df.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1 ]
print(to_del)
# drop useless columns
df.drop(df.columns[to_del], axis=1, inplace=True)
print(df.shape)

(3818, 82)
[1, 2, 7, 33, 35, 36, 58, 63, 74, 75]
(3818, 72)


In [18]:
object_columns = df.select_dtypes(include=np.object).columns.to_list()

for col in object_columns:
    df[col] = df[col].fillna("Missing Data")

In [19]:
# Check object columns for remaining missing values
df.select_dtypes(include='object').isna().sum().sort_values(ascending=False)

name                            0
summary                         0
last_review                     0
first_review                    0
calendar_updated                0
amenities                       0
bed_type                        0
room_type                       0
property_type                   0
smart_location                  0
zipcode                         0
state                           0
city                            0
neighbourhood_group_cleansed    0
neighbourhood_cleansed          0
neighbourhood                   0
street                          0
host_verifications              0
host_neighbourhood              0
host_response_time              0
host_about                      0
host_location                   0
host_since                      0
host_name                       0
transit                         0
notes                           0
neighborhood_overview           0
description                     0
space                           0
cancellation_p

In [20]:
(df.isna().sum()/len(df)*100).sort_values(ascending=False)

id                        0.0
name                      0.0
availability_30           0.0
calendar_updated          0.0
maximum_nights            0.0
                         ... 
street                    0.0
host_identity_verified    0.0
host_has_profile_pic      0.0
host_verifications        0.0
reviews_per_month         0.0
Length: 72, dtype: float64

In [21]:
df.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,smart_location,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,Stylish Queen Anne Apartment,Missing Data,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,Missing Data,Missing Data,Missing Data,956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96.000000,100.000000,False,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",True,True,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,"Seattle, WA",47.636289,-122.371025,True,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",85,567,825,254,61,2,5,1,365,4 weeks ago,14,41,71,346,207,2011-11-01,2016-01-02,95.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,False,moderate,False,False,2,4.070000
1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98.000000,100.000000,True,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",True,True,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,"Seattle, WA",47.639123,-122.365666,True,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",150,567,825,100,40,1,0,2,90,today,13,13,16,291,43,2013-08-19,2015-12-29,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,False,strict,True,True,6,1.480000
2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,16708587,Jill,2014-06-12,"Seattle, Washington, United States",i love living in Seattle. i grew up in the mi...,within a few hours,67.000000,100.000000,False,Queen Anne,2.0,2.0,"['email', 'phone', 'google', 'reviews', 'jumio']",True,True,"West Lee Street, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,"Seattle, WA",47.629724,-122.369483,True,House,Entire home/apt,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",975,567,825,254,300,10,25,4,30,5 weeks ago,1,6,17,220,20,2014-07-30,2015-09-03,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,False,strict,False,False,2,1.150000
3,7421

## Encoding Categorical Data

In [22]:
from sklearn.preprocessing import OneHotEncoder
# define one hot encoding
encoder = OneHotEncoder(sparse=False)
# transform data
onehot = encoder.fit_transform(boolean_columns)
print(onehot)
print(df.shape)

[[1. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]]
(3818, 72)


## Statistics

In [23]:
df.corr()['price']

id                                 -0.046238
host_id                            -0.049306
host_response_rate                 -0.012877
host_acceptance_rate                0.004908
host_is_superhost                   0.014249
host_listings_count                 0.095661
host_total_listings_count           0.095661
host_has_profile_pic               -0.018257
host_identity_verified              0.004247
latitude                           -0.012257
longitude                          -0.105445
is_location_exact                   0.015372
accommodates                        0.659502
bathrooms                           0.523577
bedrooms                            0.633405
beds                                0.596054
price                               1.000000
weekly_price                        0.202670
monthly_price                       0.007960
security_deposit                    0.243183
cleaning_fee                        0.591797
guests_included                     0.399087
extra_peop

In [24]:
X = df[['accommodates', 'bathrooms', 'bedrooms', 'beds','cleaning_fee', 'guests_included','weekly_price','security_deposit','extra_people']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# Four Steps
# instantiate your model
# fit model to training data
# predict on test data
# score your model on the test

lm_model = LinearRegression(normalize=True)


In [25]:
# fit model to training data
lm_model.fit(X_train, y_train)

LinearRegression(normalize=True)

In [26]:
# predict on test data
y_pred = lm_model.predict(X_test)

In [27]:
# score your model on the test
print(r2_score(y_test, y_pred))

0.533143787983573


In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from math import sqrt

cv = KFold(n_splits=10, random_state=42, shuffle=True)
classifier_pipeline = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=10))
y_pred = cross_val_predict(classifier_pipeline, X, y, cv=cv)
print("RMSE: " + str(round(sqrt(mean_squared_error(y, y_pred)), 2)))
print("R_squared: " + str(round(r2_score(y, y_pred),2)))

RMSE: 58.86
R_squared: 0.56
